# SQL Basics for Data Science — Hands-on Jupyter Notebook (Intern Level)

## Introduction
This notebook introduces foundational SQL concepts for data science using an e-commerce dataset. You'll learn to write SQL queries to analyze customer behavior, product sales, and order trends. Each section includes explanations, runnable queries, and exercises with solutions. Run cells in order to follow along.

**How to run this notebook**:
- Install dependencies: `pip install pandas jupyter sqlite3`
- Start Jupyter: `jupyter notebook`
- Run cells sequentially using Shift+Enter.

**Dataset Overview**:
- **customers**: customer_id, first_name, last_name, email, signup_date, country
- **products**: product_id, product_name, category, price
- **orders**: order_id, customer_id, order_date, status
- **order_items**: order_item_id, order_id, product_id, quantity, unit_price

**Schema Diagram** (ASCII):
```
customers(customer_id) ---- orders(customer_id)
                               |
                           order_items(order_id)
                               |
                           products(product_id)
```

In [1]:
# Setup and Data Loading
import sqlite3
import pandas as pd

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.executescript('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    signup_date TEXT,
    country TEXT
);
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price REAL
);
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    status TEXT
);
CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    unit_price REAL
);
''')

# Insert sample data
customers_data = [
    (1, 'John', 'Doe', 'john.doe@email.com', '2023-01-15', 'USA'),
    (2, 'Jane', 'Smith', 'jane.smith@email.com', '2023-02-10', 'UK'),
    (3, 'Alice', 'Brown', 'alice.brown@email.com', '2023-03-01', 'Canada'),
    (4, 'Bob', 'Wilson', 'bob.wilson@email.com', '2023-04-20', 'USA'),
    (5, 'Emma', 'Davis', 'emma.davis@email.com', '2023-05-15', 'Australia'),
    (6, 'Liam', 'Moore', 'liam.moore@email.com', '2023-06-01', 'UK'),
    (7, 'Olivia', 'Taylor', 'olivia.taylor@email.com', '2023-07-10', 'Canada'),
    (8, 'Noah', 'Anderson', 'noah.anderson@email.com', '2023-08-05', 'USA'),
    (9, 'Sophia', 'Martinez', 'sophia.martinez@email.com', '2023-09-01', 'Australia'),
    (10, 'William', 'Thomas', 'william.thomas@email.com', '2023-10-15', 'UK')
]
products_data = [
    (1, 'Laptop', 'Electronics', 999.99),
    (2, 'Smartphone', 'Electronics', 699.99),
    (3, 'Headphones', 'Electronics', 89.99),
    (4, 'T-Shirt', 'Clothing', 19.99),
    (5, 'Jeans', 'Clothing', 49.99),
    (6, 'Sneakers', 'Clothing', 79.99),
    (7, 'Backpack', 'Accessories', 39.99),
    (8, 'Watch', 'Accessories', 149.99),
    (9, 'Sunglasses', 'Accessories', 99.99),
    (10, 'Mouse', 'Electronics', 29.99)
]
orders_data = [
    (1, 1, '2023-11-01', 'completed'),
    (2, 2, '2023-11-05', 'completed'),
    (3, 3, '2023-11-10', 'cancelled'),
    (4, 4, '2023-11-15', 'completed'),
    (5, 5, '2023-11-20', 'pending'),
    (6, 1, '2023-12-01', 'completed'),
    (7, 6, '2023-12-05', 'completed'),
    (8, 7, '2023-12-10', 'cancelled'),
    (9, 8, '2023-12-15', 'completed'),
    (10, 2, '2023-12-20', 'completed')
]
order_items_data = [
    (1, 1, 1, 1, 999.99),
    (2, 1, 3, 2, 89.99),
    (3, 2, 4, 3, 19.99),
    (4, 2, 7, 1, 39.99),
    (5, 3, 2, 1, 699.99),
    (6, 4, 5, 2, 49.99),
    (7, 4, 6, 1, 79.99),
    (8, 6, 8, 1, 149.99),
    (9, 7, 9, 2, 99.99),
    (10, 9, 10, 3, 29.99),
    (11, 10, 4, 5, 19.99),
    (12, 10, 3, 1, 89.99)
]

cursor.executemany('INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?)', customers_data)
cursor.executemany('INSERT INTO products VALUES (?, ?, ?, ?)', products_data)
cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?)', orders_data)
cursor.executemany('INSERT INTO order_items VALUES (?, ?, ?, ?, ?)', order_items_data)
conn.commit()

# Sanity check: Count rows in each table
for table in ['customers', 'products', 'orders', 'order_items']:
    df = pd.read_sql_query(f'SELECT COUNT(*) AS count FROM {table}', conn)
    print(f'{table}: {df.iloc[0, 0]} rows')

customers: 10 rows
products: 10 rows
orders: 10 rows
order_items: 12 rows


## SELECT Basics
**Explanation**: The `SELECT` statement retrieves data from a table. Use `DISTINCT` to get unique values, `LIMIT` to restrict rows, and aliases (`AS`) for readable column names.

**Example**: Select product names and prices, limit to 3 rows.

In [3]:
# Select product names and prices with a limit
query = '''
SELECT product_name AS name, price AS unit_price
FROM products
LIMIT 3;
'''
pd.read_sql_query(query, conn)

,name,unit_price
0,Laptop,999.99
1,Smartphone,699.99
2,Headphones,89.99


**Exercise**: Write a query to list unique countries from the customers table.

In [4]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [5]:
# Solution: Unique countries
query = '''
SELECT DISTINCT country
FROM customers;
'''
pd.read_sql_query(query, conn)

,country
0,USA
1,UK
2,Canada
3,Australia


## Filtering with WHERE
**Explanation**: `WHERE` filters rows based on conditions. Use operators like `=`, `>`, `<`, `BETWEEN`, `IN`, `LIKE`, `IS NULL`.

**Example**: Find customers from the USA who signed up after March 2023.

In [6]:
# Filter customers by country and signup date
query = '''
SELECT first_name, last_name, signup_date
FROM customers
WHERE country = 'USA' AND signup_date > '2023-03-01';
'''
pd.read_sql_query(query, conn)

,first_name,last_name,signup_date
0,Bob,Wilson,2023-04-20
1,Noah,Anderson,2023-08-05


**Exercise**: Write a query to find customers from Canada who signed up after April 2023.

In [7]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [8]:
# Solution: Customers from Canada after April 2023
query = '''
SELECT first_name, last_name, signup_date
FROM customers
WHERE country = 'Canada' AND signup_date > '2023-04-01';
'''
pd.read_sql_query(query, conn)

,first_name,last_name,signup_date
0,Olivia,Taylor,2023-07-10


## Sorting with ORDER BY and Limiting Results
**Explanation**: `ORDER BY` sorts results (ASC or DESC). Combine with `LIMIT` to get top/bottom rows.

**Example**: Find the top 3 most expensive products.

In [9]:
# Top 3 most expensive products
query = '''
SELECT product_name, price
FROM products
ORDER BY price DESC
LIMIT 3;
'''
pd.read_sql_query(query, conn)

,product_name,price
0,Laptop,999.99
1,Smartphone,699.99
2,Watch,149.99


**Exercise**: Write a query to find the 5 cheapest products, sorted by price ascending.

In [10]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [11]:
# Solution: 5 cheapest products
query = '''
SELECT product_name, price
FROM products
ORDER BY price ASC
LIMIT 5;
'''
pd.read_sql_query(query, conn)

,product_name,price
0,T-Shirt,19.99
1,Mouse,29.99
2,Backpack,39.99
3,Jeans,49.99
4,Sneakers,79.99


## JOINs (INNER and LEFT)
**Explanation**: `INNER JOIN` matches rows between tables; `LEFT JOIN` keeps all rows from the left table. Use to combine data across tables.

**Example**: Calculate revenue per order item (quantity * unit_price).

In [12]:
# Revenue per order item
query = '''
SELECT oi.order_id, p.product_name, oi.quantity, oi.unit_price, (oi.quantity * oi.unit_price) AS revenue
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id;
'''
pd.read_sql_query(query, conn)

,order_id,product_name,quantity,unit_price,revenue
0,1,Laptop,1,999.99,999.99
1,1,Headphones,2,89.99,179.98
2,2,T-Shirt,3,19.99,59.97
3,2,Backpack,1,39.99,39.99
4,3,Smartphone,1,699.99,699.99
5,4,Jeans,2,49.99,99.98
6,4,Sneakers,1,79.99,79.99
7,6,Watch,1,149.99,149.99
8,7,Sunglasses,2,99.99,199.98
9,9,Mouse,3,29.99,89.97


**Exercise**: List all customers and their last completed order date (use LEFT JOIN).

In [13]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [14]:
# Solution: Customers with last completed order date
query = '''
SELECT c.first_name, c.last_name, MAX(o.order_date) AS last_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE o.status = 'completed' OR o.order_date IS NULL
GROUP BY c.customer_id, c.first_name, c.last_name;
'''
pd.read_sql_query(query, conn)

,first_name,last_name,last_order_date
0,John,Doe,2023-12-01
1,Jane,Smith,2023-12-20
2,Bob,Wilson,2023-11-15
3,Liam,Moore,2023-12-05
4,Noah,Anderson,2023-12-15
5,Sophia,Martinez,None
6,William,Thomas,None


## Aggregations and GROUP BY
**Explanation**: `GROUP BY` groups rows for aggregation. Use `COUNT`, `SUM`, `AVG`, `MIN`, `MAX` to summarize data.

**Example**: Calculate total revenue by product category for completed orders.

In [15]:
# Revenue by product category
query = '''
SELECT p.category, SUM(oi.quantity * oi.unit_price) AS total_revenue
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id
INNER JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'completed'
GROUP BY p.category;
'''
pd.read_sql_query(query, conn)

,category,total_revenue
0,Accessories,389.96
1,Clothing,339.89
2,Electronics,1359.93


**Exercise**: Calculate total quantity sold per product.

In [16]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [17]:
# Solution: Total quantity per product
query = '''
SELECT p.product_name, SUM(oi.quantity) AS total_quantity
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id
INNER JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'completed'
GROUP BY p.product_name;
'''
pd.read_sql_query(query, conn)

,product_name,total_quantity
0,Backpack,1
1,Headphones,3
2,Jeans,2
3,Laptop,1
4,Mouse,3
5,Sneakers,1
6,Sunglasses,2
7,T-Shirt,8
8,Watch,1


## HAVING
**Explanation**: `HAVING` filters grouped results, used after `GROUP BY`.

**Example**: Find categories with total revenue > $500.

In [18]:
# Categories with revenue > $500
query = '''
SELECT p.category, SUM(oi.quantity * oi.unit_price) AS total_revenue
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id
INNER JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'completed'
GROUP BY p.category
HAVING total_revenue > 500;
'''
pd.read_sql_query(query, conn)

,category,total_revenue
0,Electronics,1359.93


**Exercise**: Find countries with at least 2 completed orders and average order value > $100.

In [19]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [20]:
# Solution: Countries with >= 2 orders and avg order value > $100
query = '''
SELECT c.country, COUNT(o.order_id) AS order_count, AVG(oi.quantity * oi.unit_price) AS avg_order_value
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.status = 'completed'
GROUP BY c.country
HAVING order_count >= 2 AND avg_order_value > 100;
'''
pd.read_sql_query(query, conn)

,country,order_count,avg_order_value
0,USA,6,266.65


## Basic Subqueries
**Explanation**: Subqueries are queries within queries, used in `SELECT` or `WHERE` to compute values.

**Example**: Find customers with total spend above the overall average.

In [21]:
# Customers with above-average spend
query = '''
SELECT c.first_name, c.last_name, SUM(oi.quantity * oi.unit_price) AS total_spend
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.status = 'completed'
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING total_spend > (
    SELECT AVG(total) FROM (
        SELECT SUM(oi.quantity * oi.unit_price) AS total
        FROM orders o
        INNER JOIN order_items oi ON o.order_id = oi.order_id
        WHERE o.status = 'completed'
        GROUP BY o.customer_id
    ) sub
);
'''
pd.read_sql_query(query, conn)

,first_name,last_name,total_spend
0,John,Doe,1329.96


**Exercise**: Find products priced above their category's average price.

In [22]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [23]:
# Solution: Products above category average price
query = '''
SELECT p.product_name, p.category, p.price
FROM products p
WHERE p.price > (
    SELECT AVG(price)
    FROM products p2
    WHERE p2.category = p.category
);
'''
pd.read_sql_query(query, conn)

,product_name,category,price
0,Laptop,Electronics,999.99
1,Smartphone,Electronics,699.99
2,Sneakers,Clothing,79.99
3,Watch,Accessories,149.99
4,Sunglasses,Accessories,99.99


## Practical Analysis Scenarios
**Explanation**: Combine SQL techniques to answer business questions, like customer lifetime value (CLV), order frequency, and trends.

**Example**: Calculate customer lifetime value (proxy: total spend) and order frequency.

In [24]:
# Customer lifetime value and order frequency
query = '''
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    COUNT(DISTINCT o.order_id) AS order_count,
    SUM(oi.quantity * oi.unit_price) AS clv
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.status = 'completed' OR o.order_id IS NULL
GROUP BY c.customer_id, c.first_name, c.last_name;
'''
pd.read_sql_query(query, conn)

,customer_id,first_name,last_name,order_count,clv
0,1,John,Doe,2,1329.96
1,2,Jane,Smith,2,289.90
2,4,Bob,Wilson,1,179.97
3,6,Liam,Moore,1,199.98
4,8,Noah,Anderson,1,89.97
5,9,Sophia,Martinez,0,NaN
6,10,William,Thomas,0,NaN


**Exercise**: Calculate monthly signup cohorts with their completed order counts.

In [25]:
# Your query here
query = '''
-- Write your query below

'''
pd.read_sql_query(query, conn)

TypeError: 'NoneType' object is not iterable

In [26]:
# Solution: Monthly signup cohorts with order counts
query = '''
SELECT 
    strftime('%Y-%m', c.signup_date) AS signup_month,
    COUNT(DISTINCT c.customer_id) AS customer_count,
    COUNT(DISTINCT o.order_id) AS order_count
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE o.status = 'completed' OR o.order_id IS NULL
GROUP BY signup_month;
'''
pd.read_sql_query(query, conn)

,signup_month,customer_count,order_count
0,2023-01,1,2
1,2023-02,1,2
2,2023-04,1,1
3,2023-06,1,1
4,2023-08,1,1
5,2023-09,1,0
6,2023-10,1,0


## Common Pitfalls and Tips
- **NULL Handling**: Use `IS NULL` or `IS NOT NULL` to check for NULLs. `WHERE column = NULL` doesn't work.
- **Duplicates**: Use `DISTINCT` to avoid duplicate rows, or ensure joins don't multiply rows.
- **INNER vs LEFT JOIN**: INNER JOIN excludes unmatched rows; LEFT JOIN keeps all rows from the left table.
- **DISTINCT vs GROUP BY**: Use `DISTINCT` for unique values in a single column; `GROUP BY` for aggregations.

**Example**: Check for NULLs in customer emails.

In [27]:
# Check for NULL emails
query = '''
SELECT first_name, last_name, email
FROM customers
WHERE email IS NULL;
'''
pd.read_sql_query(query, conn)

,first_name,last_name,email


## Wrap-up and Further Reading
You've learned SQL basics: SELECT, WHERE, ORDER BY, LIMIT, DISTINCT, JOINs, GROUP BY, HAVING, and subqueries. Practice by exploring the dataset further or trying these resources:
- SQLite documentation: https://www.sqlite.org/docs.html
- SQL for Data Analysis (book): "SQL for Data Scientists" by Renee M. P. Teate
- Online practice: Mode Analytics SQL Tutorial (https://mode.com/sql-tutorial/)

**Optional Mini-Project**: Analyze the dataset to answer:
1. Which customers have spent the most?
2. What is the average order value by country?
3. Which products are most popular (by quantity sold)?
4. Visualize monthly revenue trends (requires pandas/plotly).

In [28]:
# Optional: Monthly revenue trend visualization
import plotly.express as px

query = '''
SELECT strftime('%Y-%m', o.order_date) AS month, SUM(oi.quantity * oi.unit_price) AS revenue
FROM orders o
INNER JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.status = 'completed'
GROUP BY month;
'''
df = pd.read_sql_query(query, conn)
fig = px.line(df, x='month', y='revenue', title='Monthly Revenue Trend')
fig.show()

ModuleNotFoundError: No module named 'plotly'